AISD Coursework - Part B (Proxy training for Indian context) - PyTorch Version

File: myaisd_cw2_part2.py

This script implements a lightweight Attention U-Net in PyTorch and trains it on
the LandCover.ai v1 dataset (5 classes: other, building, woodland, water, road).

It covers:

- Part A, Task 3:
  * Identify a contextually relevant multi-class land-cover dataset.
  * Provide a full preprocessing pipeline (large orthophotos → 512x512 tiles).

- Part A, Task 4:
  * Adapt the model architecture to a multi-class Attention U-Net (AttUNet).
  * Implement a custom mIoU metric for multi-class segmentation.

- Part A, Task 5:
  * Train the adapted model for 12 epochs.
  * Track training/validation loss and mean IoU.
  * Save the best model checkpoint based on validation mIoU.

This script is the fully-completed proxy training run for the Indian
context when GPU limits prevented running the 25-epoch TensorFlow version
in AISD_CW2_partB.py end-to-end.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# SETUP: Mount Drive, install deps, set device & seeds

!pip install rasterio==1.3.10

import os
import random
import numpy as np
import torch

# Set a base working directory (change if you like)
BASE_DIR = "/content"
DATA_DIR = os.path.join(BASE_DIR, "data_landcoverai")
TILES_DIR = os.path.join(DATA_DIR, "tiles_512")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(TILES_DIR, exist_ok=True)

# Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
!gdown 1C4M9NLmKMluSleqrs9jwELyhl725LLAN -O landcover_ai.zip
!unzip landcover_ai.zip -d landcover_ai


Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1C4M9NLmKMluSleqrs9jwELyhl725LLAN

but Gdown can't. Please check connections and permissions.
unzip:  cannot find or open landcover_ai.zip, landcover_ai.zip.zip or landcover_ai.zip.ZIP.


In [ ]:
!ls -lh /content/data_landcoverai/raw


ls: cannot access '/content/data_landcoverai/raw': No such file or directory


In [ ]:
import zipfile, os

zip_path = "/content/data_landcoverai/raw/landcover_ai_v1.zip"
extract_dir = "/content/data_landcoverai/raw/landcover_ai_v1"

if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(extract_dir)
    print("Extracted successfully to:", extract_dir)
else:
    print("Already extracted:", extract_dir)

!ls -R "/content/data_landcoverai/raw/landcover_ai_v1"


FileNotFoundError: [Errno 2] No such file or directory: '/content/data_landcoverai/raw/landcover_ai_v1.zip'

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive"))


['Colab Notebooks', 'PartB_AttUNet_LandCoverAI_best.pth', 'landcoverai_saved']


In [ ]:
extract_dir


'/content/data_landcoverai/raw/landcover_ai_v1'

In [ ]:
IMGS_DIR = os.path.join(extract_dir, "images")
MASKS_DIR = os.path.join(extract_dir, "masks")

print("IMGS_DIR =", IMGS_DIR)
print("MASKS_DIR =", MASKS_DIR)

!ls -l "$IMGS_DIR"
!ls -l "$MASKS_DIR"


IMGS_DIR = /content/data_landcoverai/raw/landcover_ai_v1/images
MASKS_DIR = /content/data_landcoverai/raw/landcover_ai_v1/masks
ls: cannot access '/content/data_landcoverai/raw/landcover_ai_v1/images': No such file or directory
ls: cannot access '/content/data_landcoverai/raw/landcover_ai_v1/masks': No such file or directory


In [ ]:
import os, glob, numpy as np
from PIL import Image
import rasterio
from rasterio.windows import Window
from tqdm import tqdm

# Correct paths
IMGS_DIR = os.path.join(extract_dir, "images")
MASKS_DIR = os.path.join(extract_dir, "masks")

print("IMGS_DIR =", IMGS_DIR)
print("MASKS_DIR =", MASKS_DIR)

tiles_img_dir = os.path.join(TILES_DIR, "images")
tiles_msk_dir = os.path.join(TILES_DIR, "masks")

os.makedirs(tiles_img_dir, exist_ok=True)
os.makedirs(tiles_msk_dir, exist_ok=True)

img_files = sorted(glob.glob(os.path.join(IMGS_DIR, "*.tif")))
print("Found", len(img_files), "orthophotos.")

tile_size = 512
tile_count = 0

for img_path in tqdm(img_files, desc="Tiling orthophotos"):
    fname = os.path.splitext(os.path.basename(img_path))[0]
    mask_path = os.path.join(MASKS_DIR, fname + ".tif")

    if not os.path.exists(mask_path):
        print("Missing mask for", fname)
        continue

    with rasterio.open(img_path) as src_img, rasterio.open(mask_path) as src_msk:
        H, W = src_img.height, src_img.width
        nx = W // tile_size
        ny = H // tile_size

        for iy in range(ny):
            for ix in range(nx):
                window = Window(ix * tile_size, iy * tile_size, tile_size, tile_size)

                img_tile = src_img.read(window=window)
                msk_tile = src_msk.read(1, window=window)

                img_tile = np.transpose(img_tile, (1,2,0))
                img_tile = np.clip(img_tile, 0,255).astype(np.uint8)
                msk_tile = msk_tile.astype(np.uint8)

                tile_id = f"{fname}_{iy}_{ix}"
                Image.fromarray(img_tile).save(os.path.join(tiles_img_dir, tile_id+".png"))
                Image.fromarray(msk_tile).save(os.path.join(tiles_msk_dir, tile_id+".png"))

                tile_count += 1

print("Total tiles generated:", tile_count)
print("Tile images:", len(glob.glob(os.path.join(tiles_img_dir,'*.png'))))
print("Tile masks:", len(glob.glob(os.path.join(tiles_msk_dir,'*.png'))))



IMGS_DIR = /content/data_landcoverai/raw/landcover_ai_v1/images
MASKS_DIR = /content/data_landcoverai/raw/landcover_ai_v1/masks
Found 0 orthophotos.


Tiling orthophotos: 0it [00:00, ?it/s]

Total tiles generated: 0
Tile images: 0
Tile masks: 0


In [ ]:
# ============================================================
# B. PyTorch Dataset & DataLoaders
# ============================================================

import glob
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

tiles_img_dir = "/content/data_landcoverai/tiles_512/images"
tiles_msk_dir = "/content/data_landcoverai/tiles_512/masks"

# Collect aligned image-mask pairs
all_imgs = sorted(glob.glob(os.path.join(tiles_img_dir, "*.png")))
all_msks = sorted(glob.glob(os.path.join(tiles_msk_dir, "*.png")))

print("Total PNG tiles:", len(all_imgs), len(all_msks))

# Subsample to 3600 (3000 train, 600 val)
max_total = 3600
if len(all_imgs) > max_total:
    idx = np.random.choice(len(all_imgs), max_total, replace=False)
    all_imgs = [all_imgs[i] for i in idx]
    all_msks = [all_msks[i] for i in idx]
    print(f"Subsampled to {max_total} tiles.")

# Train/Val split (3000/600)
train_imgs, val_imgs, train_msks, val_msks = train_test_split(
    all_imgs, all_msks, test_size=600, random_state=42
)

print("Train samples:", len(train_imgs))
print("Val samples:", len(val_imgs))

# Image normalization
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

def mask_to_tensor(mask_img):
    mask_np = np.array(mask_img, dtype=np.int64)
    return torch.from_numpy(mask_np)

class LandCoverAIDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("RGB")
        msk = Image.open(self.masks[idx])

        img = img_transform(img)
        msk = mask_to_tensor(msk)

        return img, msk

train_ds = LandCoverAIDataset(train_imgs, train_msks)
val_ds   = LandCoverAIDataset(val_imgs, val_msks)

train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=2, pin_memory=True)

print("Dataloaders ready.")

Total PNG tiles: 0 0


ValueError: test_size=600 should be either positive and smaller than the number of samples 0 or a float in the (0, 1) range

In [ ]:
# C. Lightweight Attention U-Net (PyTorch)

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        # g: gating (decoder), x: skip (encoder)
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

class UpBlockAttn(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        self.attn = AttentionBlock(F_g=out_ch, F_l=skip_ch, F_int=out_ch // 2)
        self.conv = DoubleConv(out_ch + skip_ch, out_ch)

    def forward(self, x, skip):
        x = self.up(x)
        # Pad if needed (for odd dims)
        diffY = skip.size()[2] - x.size()[2]
        diffX = skip.size()[3] - x.size()[3]
        x = F.pad(x, [diffX // 2, diffX - diffX // 2,
                      diffY // 2, diffY - diffY // 2])

        skip = self.attn(x, skip)
        x = torch.cat([skip, x], dim=1)
        return self.conv(x)

class AttUNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=5, base_ch=32):
        super().__init__()
        self.inc = DoubleConv(n_channels, base_ch)

        self.down1 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(base_ch, base_ch * 2)
        )
        self.down2 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(base_ch * 2, base_ch * 4)
        )
        self.down3 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(base_ch * 4, base_ch * 8)
        )
        self.down4 = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(base_ch * 8, base_ch * 16)
        )

        self.up1 = UpBlockAttn(base_ch * 16, base_ch * 8, base_ch * 8)
        self.up2 = UpBlockAttn(base_ch * 8,  base_ch * 4, base_ch * 4)
        self.up3 = UpBlockAttn(base_ch * 4,  base_ch * 2, base_ch * 2)
        self.up4 = UpBlockAttn(base_ch * 2,  base_ch,     base_ch)

        self.outc = nn.Conv2d(base_ch, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)        # base
        x2 = self.down1(x1)     # 2*base
        x3 = self.down2(x2)     # 4*base
        x4 = self.down3(x3)     # 8*base
        x5 = self.down4(x4)     # 16*base

        x = self.up1(x5, x4)
        x = self.up2(x,  x3)
        x = self.up3(x,  x2)
        x = self.up4(x,  x1)

        logits = self.outc(x)
        return logits

# LandCover.ai v1 has 5 classes: 0 other, 1 building, 2 woodland, 3 water, 4 road
n_classes = 5

# If i'll use CUDA OOM later, change base_ch=32, base_ch=16
model = AttUNet(n_channels=3, n_classes=n_classes, base_ch=32).to(device)

total_params = sum(p.numel() for p in model.parameters()) / 1e6
print(f"Model built. Total parameters: {total_params:.2f}M")

Using device: cpu
Model built. Total parameters: 7.85M


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

n_classes = 5
best_model_path = "/content/drive/MyDrive/PartB_AttUNet_LandCoverAI_best.pth"

# Rebuild EXACTLY the same architecture
model = AttUNet(n_channels=3, n_classes=n_classes, base_ch=32).to(device)

# Load weights
state = torch.load(best_model_path, map_location=device)
model.load_state_dict(state)
model.eval()

print("Loaded best model from:", best_model_path)


Using device: cpu


NameError: name 'AttUNet' is not defined

In [ ]:
# D. Training loop (12 epochs, save best model to Drive)

from tqdm import tqdm
import numpy as np

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# FIXED: remove verbose argument
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=2
)

def compute_iou(pred, target, num_classes):
    pred = pred.view(-1)
    target = target.view(-1)
    ious = []
    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds & target_inds).sum().item()
        union = (pred_inds | target_inds).sum().item()
        if union == 0:
            continue
        ious.append(intersection / union)
    if len(ious) == 0:
        return 0.0
    return float(np.mean(ious))

best_val_iou = 0.0
best_model_path = "/content/drive/MyDrive/PartB_AttUNet_LandCoverAI_best.pth"

epochs = 12

for epoch in range(1, epochs + 1):
    # ---------- TRAIN ----------
    model.train()
    train_loss = 0.0
    train_iou = 0.0
    num_batches = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs} [Train]")
    for imgs, masks in pbar:
        imgs = imgs.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(dim=1)
        train_iou += compute_iou(preds.cpu(), masks.cpu(), n_classes)
        num_batches += 1

        pbar.set_postfix({
            "loss": train_loss / num_batches,
            "mIoU": train_iou / num_batches
        })

    avg_train_loss = train_loss / num_batches
    avg_train_iou = train_iou / num_batches

    # ---------- VALIDATION ----------
    model.eval()
    val_loss = 0.0
    val_iou = 0.0
    num_val_batches = 0

    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch}/{epochs} [Val]"):
            imgs = imgs.to(device, non_blocking=True)
            masks = masks.to(device, non_blocking=True)

            outputs = model(imgs)
            loss = criterion(outputs, masks)

            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            val_iou += compute_iou(preds.cpu(), masks.cpu(), n_classes)
            num_val_batches += 1

    avg_val_loss = val_loss / num_val_batches
    avg_val_iou = val_iou / num_val_batches

    scheduler.step(avg_val_iou)

    print(
        f"Epoch {epoch}/{epochs} | "
        f"TrainLoss={avg_train_loss:.4f}, Train mIoU={avg_train_iou:.4f} | "
        f"ValLoss={avg_val_loss:.4f}, Val mIoU={avg_val_iou:.4f}"
    )

    # Save best model
    if avg_val_iou > best_val_iou:
        best_val_iou = avg_val_iou
        torch.save(model.state_dict(), best_model_path)
        print(f"  >> New best model saved with Val mIoU = {best_val_iou:.4f}")

print("Training finished.")
print("Best Val mIoU:", best_val_iou)
print("Best model path:", best_model_path)

Epoch 1/12 [Val]: 100%|██████████| 300/300 [00:23<00:00, 12.98it/s]


Epoch 1/12 | TrainLoss=0.8414, Train mIoU=0.3312 | ValLoss=0.7230, Val mIoU=0.2933
  >> New best model saved with Val mIoU = 0.2933


Epoch 2/12 [Val]: 100%|██████████| 300/300 [00:22<00:00, 13.38it/s]


Epoch 2/12 | TrainLoss=0.7269, Train mIoU=0.3549 | ValLoss=0.6178, Val mIoU=0.3940
  >> New best model saved with Val mIoU = 0.3940


Epoch 3/12 [Val]: 100%|██████████| 300/300 [00:22<00:00, 13.39it/s]


...

Epoch 10/12 | TrainLoss=0.4369, Train mIoU=0.3503 | ValLoss=0.5339, Val mIoU=0.3517


Epoch 11/12 [Val]: 100%|██████████| 300/300 [00:22<00:00, 13.16it/s]


Epoch 11/12 | TrainLoss=0.4270, Train mIoU=0.3565 | ValLoss=0.6046, Val mIoU=0.2879


Epoch 12/12 [Val]: 100%|██████████| 300/300 [00:22<00:00, 13.09it/s]

Epoch 12/12 | TrainLoss=0.4092, Train mIoU=0.3563 | ValLoss=0.6379, Val mIoU=0.2825
Training finished.
Best Val mIoU: 0.393999537232429
Best model path: /content/drive/MyDrive/PartB_AttUNet_LandCoverAI_best.pth


In [ ]:
#from sentinelhub import SHConfig

#config = SHConfig()

#config.sh_client_id = "d81449d9-0e75-4df1-8d63-10778ce9e07d"
#config.sh_client_secret = "d81449d9-0e75-4df1-8d63-10778ce9e07d"

#config.save()   # saves into ~/.config/sentinelhub/config.json

#print("ID =", config.sh_client_id)
#print("Secret =", config.sh_client_secret)

#not using htis approahc as too many complexities arising with joshimath dataset fetch

In [ ]:
!pip install torchgeo
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

from torchgeo.datasets import LandCoverAI

root_dir = "/content/landcoverai"

dataset = LandCoverAI(
    root=root_dir,
    split="train",
    download=True
)

print("Dataset size:", len(dataset))

100%|██████████| 1.54G/1.54G [01:42<00:00, 15.1MB/s]


Processed M-33-20-D-c-4-2 1/41
Processed M-33-20-D-d-3-3 2/41
Processed M-33-32-B-b-4-4 3/41
Processed M-33-48-A-c-4-4 4/41
Processed M-33-7-A-d-2-3 5/41
Processed M-33-7-A-d-3-2 6/41
Processed M-34-32-B-a-4-3 7/41
Processed M-34-32-B-b-1-3 8/41
Processed M-34-5-D-d-4-2 9/41
Processed M-34-51-C-b-2-1 10/41
Processed M-34-51-C-d-4-1 11/41
Processed M-34-55-B-b-4-1 12/41
Processed M-34-56-A-b-1-4 13/41
Processed M-34-6-A-d-2-2 14/41
Processed M-34-65-D-a-4-4 15/41
Processed M-34-65-D-c-4-2 16/41
Processed M-34-65-D-d-4-1 17/41
Processed M-34-68-B-a-1-3 18/41
Processed M-34-77-B-c-2-3 19/41
Processed N-33-104-A-c-1-1 20/41
Processed N-33-119-C-c-3-3 21/41
Processed N-33-130-A-d-3-3 22/41
Processed N-33-130-A-d-4-4 23/41
Processed N-33-139-C-d-2-2 24/41
Processed N-33-139-C-d-2-4 25/41
Processed N-33-139-D-c-1-3 26/41
Processed N-33-60-D-c-4-2 27/41
Processed N-33-60-D-d-1-2 28/41
Processed N-33-96-D-d-1-1 29/41
Processed N-34-106-A-b-3-4 30/41
Processed N-34-106-A-c-1-3 31/41
Processed N-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/landcoverai_saved
!cp -r /content/landcoverai/* /content/drive/MyDrive/landcoverai_saved/

print("Dataset copied to Drive successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/landcoverai/*': No such file or directory
Dataset copied to Drive successfully!


In [ ]:
from torchgeo.datasets import LandCoverAI

root_dir = "/content/drive/MyDrive/landcoverai_saved"

train_ds = LandCoverAI(root=root_dir, split="train", download=False)
val_ds   = LandCoverAI(root=root_dir, split="val", download=False)
test_ds  = LandCoverAI(root=root_dir, split="test", download=False)

print("Train:", len(train_ds))
print("Val:", len(val_ds))
print("Test:", len(test_ds))

Train: 7470
Val: 1602
Test: 1602


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=2)

print("Dataloaders ready!")


Dataloaders ready!


In [ ]:
import torch

best_model_path = "/content/drive/MyDrive/PartB_AttUNet_LandCoverAI_best.pth"

model = AttUNet(n_channels=3, n_classes=5, base_ch=32)
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.to(device)
model.eval()

print("Loaded trained AttUNet model successfully!")

Loaded trained AttUNet model successfully!


In [ ]:
import numpy as np

def compute_iou(pred, true, num_classes=5):
    ious = []
    for cls in range(num_classes):
        pred_inds = (pred == cls)
        true_inds = (true == cls)

        intersection = (pred_inds & true_inds).sum()
        union = (pred_inds | true_inds).sum()

        if union == 0:
            ious.append(np.nan)
        else:
            ious.append(intersection / union)

    return ious


def evaluate_model(model, loader, num_classes=5):
    model.eval()
    iou_list = []
    pixel_acc_list = []

    with torch.no_grad():
        for batch in loader:
            img = batch["image"].to(device)
            mask = batch["mask"].to(device)

            logits = model(img)
            preds = torch.argmax(logits, dim=1)

            preds_np = preds.cpu().numpy().reshape(-1)
            mask_np  = mask.cpu().numpy().reshape(-1)

            # Pixel accuracy
            pixel_acc = (preds_np == mask_np).mean()
            pixel_acc_list.append(pixel_acc)

            # IoU
            ious = compute_iou(preds_np, mask_np, num_classes)
            iou_list.append(ious)

    mean_pixel_acc = np.mean(pixel_acc_list)
    mean_iou = np.nanmean(iou_list, axis=0)
    mean_mIoU = np.nanmean(mean_iou)

    return mean_pixel_acc, mean_iou, mean_mIoU

In [ ]:
def fast_loader(loader, max_batches=20):
    """Take only a small number of batches for quick evaluation."""
    for i, batch in enumerate(loader):
        if i >= max_batches:
            break
        yield batch

In [ ]:
val_acc, val_iou_per_class, val_miou = evaluate_model(
    model,
    fast_loader(val_loader, max_batches=20)
)

test_acc, test_iou_per_class, test_miou = evaluate_model(
    model,
    fast_loader(test_loader, max_batches=20)
)

print("===== FAST VALIDATION METRICS =====")
print("Pixel Accuracy:", val_acc)
print("mIoU:", val_miou)
print("Class-wise IoU:", val_iou_per_class)

print("\n===== FAST TEST METRICS =====")
print("Pixel Accuracy:", test_acc)
print("mIoU:", test_miou)
print("Class-wise IoU:", test_iou_per_class)

===== FAST VALIDATION METRICS =====
Pixel Accuracy: 0.33119850158691405
mIoU: 0.07215405794135644
Class-wise IoU: [3.44622710e-01 5.32539406e-04 2.45643959e-05 0.00000000e+00
 1.55904755e-02]

===== FAST TEST METRICS =====
Pixel Accuracy: 0.318635082244873
mIoU: 0.06991305548726745
Class-wise IoU: [3.36096302e-01 1.50123314e-04 2.35983595e-05 0.00000000e+00
 1.32952541e-02]
